## Distinguishing Failed Experiments with Rubicon 
Modeling runs can fail for a number of reasons. When logging with rubicon_ml, a failed run may result in an empty or incomplete experiment. In this example, we'll walk through how to handle such experiments.

First lets simulate the problem. To do this we'll create an estimator that will fail on it's `fit()` 30% of the time.

In [1]:
from sklearn.base import BaseEstimator
import random

class BadEstimator(BaseEstimator):
    def __init__(self, state=None):
        super().__init__()
        self.state=None
        self.knn = KNeighborsClassifier(n_neighbors=2)
    def fit(self, X, y):
        self.knn.fit(X, y)
        output=random.random()
        if output<.3:
            output=None
        self.state=output
    def score(self, X):
        knn_score = self.knn.score(X)
        return knn_score

Now let's attempt to create a `rubicon_ml.sklearn` pipeline with this failing estimator and attempt to `fit` the pipeline. 

In [2]:
from rubicon_ml.sklearn import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from rubicon_ml import Rubicon

random.seed(18)

rubicon = Rubicon(
    persistence="memory",
)
project = rubicon.get_or_create_project(name="Failed Experiment")

X = [[1], [1], [1], [1]]
y = [1, 1, 1, 1]
pipe=make_pipeline(project, SimpleImputer(strategy="mean"),BadEstimator())
for _ in range(20):
    pipe.fit(X,y)
for exp in project.experiments():
    for p in exp.parameters():
        if (p.name=="badestimator__state" and p.value==None):
            exp.add_tags(["failed"])

Finally, we can now retrieve all our failed experiments by passing the `tags=["failed"]` to `project.experiments()`.

In [3]:
for exp in project.experiments(tags=["failed"]):
    print(exp)

Experiment(project_name='Failed Experiment', id='c8a12f83-d4e1-4892-aa64-b3ee9ccb12e6', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 43, 24, 872174))
Experiment(project_name='Failed Experiment', id='c10fca95-46a5-474c-ab22-fcd592fa4ae1', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 43, 24, 876613))
Experiment(project_name='Failed Experiment', id='d82f763a-83b2-4148-b170-461cda67d83a', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 43, 24, 886114))
Experiment(project_name='Failed Experiment', id='a27f872b-2f5d-44d8-bbb0-7af387d61bc1', name='RubiconPipeline